In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_model_and_tokenizer(model_dir):
    tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}/tokenizer")
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.to(device)
    model.eval()
    return tokenizer, model

def predict_review(texts, tokenizer, model):
    if isinstance(texts, str):
        texts = [texts]

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    return preds.cpu().tolist(), probs.cpu().tolist()

examples = [
    "This product is amazing! Highly recommended.",
    "Awful! Completely broken after one use. Do not buy.",
    "I will never stay at this hotel again.",
    "Room was clean, staff were friendly and breakfast was great."
]

model_paths = [
    './fake_review_detection_model/bert-base-uncased',
    './fake_review_detection_model/roberta-base',
    './fake_review_detection_model/google/electra-base-discriminator'
]

results = {'text': examples}

for i, path in enumerate(model_paths, 1):
    tokenizer, model = load_model_and_tokenizer(path)
    labels, _ = predict_review(examples, tokenizer, model)
    results[f'model{i}'] = labels  


----------------------------------------
Review: This product is amazing! Highly recommended.
Predicted Label: 0 (Real)
Confidence: [0.653107762336731, 0.34689226746559143]

----------------------------------------
Review: Awful! Completely broken after one use. Do not buy.
Predicted Label: 0 (Real)
Confidence: [0.8366106748580933, 0.16338935494422913]

----------------------------------------
Review: I will never stay at this hotel again.
Predicted Label: 1 (Fake)
Confidence: [0.3176462948322296, 0.682353675365448]

----------------------------------------
Review: Room was clean, staff were friendly and breakfast was great.
Predicted Label: 0 (Real)
Confidence: [0.6616242527961731, 0.3383758068084717]


In [ ]:
df = pd.DataFrame(results)
print(df)